# Kannada to English Machine Translation

Importing/Downloading Libraries

In [1]:
#in case conda environment not available locally, run these commands, or open this in colab
%pip install tensorflow
%pip install nltk

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk.translate.bleu_score as bleu
import random
import string
from sklearn.model_selection import train_test_split
import os
import time

## Creating CSV Files

In [2]:
small_kn = open('./data/small.kn', encoding='utf-8').readlines()
small_en = open('./data/small.en').readlines()
for pair in list(zip(small_en[:5], small_kn[:5])):
    print(pair[0][:-2],'\t',pair[1][:-2])

Such videos are aplenty on social media channels 	 ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು ಹರಿದಾಡುತ್ತಿರುತ್ತವೆ
This incident happened within the limits of Mudhol Police Station 	 ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದುರ್ಘಟನೆ ನಡೆದಿದೆ
The situation is currently under control, he said 	 ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದು ತಿಳಿಸಿದ್ದಾರೆ
Choose a Bookmark to be your Home Page. If you choose a folder, the Bookmarks in that folder will be opened in Tabs 	 ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್ ಅನ್ನು ಆರಿಸಿ. ನೀವು ಒಂದು ಕೋಶವನ್ನು ಆರಿಸಿದರೆ, ಅದರಲ್ಲಿನ ಬುಕ್‌ಮಾರ್ಕುಗಳು ಪ್ರತ್ಯೇಕ ಹಾಳೆಗಳಾಗಿ ತೆರೆಯಲ್ಪಡುತ್ತವೆ
Several social organisations are also being involved 	 ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರಕವಾಗಿ ಕಾರ್ಯ ನಿರ್ವಹಿಸುತ್ತವೆ


In [3]:
with open('./data/kan_eng.csv','w') as f:
    for pair in list(zip(small_en, small_kn)):
        new_pair0 = pair[0].replace(',','')
        new_pair1 = pair[1].replace(',', '')
        new_pair0 = new_pair0.replace('\'', '')
        new_pair1 = new_pair1.replace('\'', '')
        new_pair0 = new_pair0.replace('\n','')
        new_pair1 = new_pair1.replace('\n','')
        str = new_pair0 + ',' + new_pair1
        f.write(str)
        f.write('\n') 

In [4]:
df = pd.read_csv('./data/kan_eng.csv',names=['English','Kannada'])

In [5]:
df.head()

,English,Kannada
0,Such videos are aplenty on social media channels.,ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು ಹರಿದಾಡು...
1,This incident happened within the limits of Mu...,ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದುರ್ಘಟನೆ ನ...
2,The situation is currently under control he said.,ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದು ತಿಳಿಸಿ...
3,Choose a Bookmark to be your Home Page. If you...,ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್ ಅನ್ನು ಆ...
4,Several social organisations are also being in...,ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರಕವಾಗಿ ಕಾ...


In [6]:
df.dropna(inplace=True)
df.shape

(250000, 2)

In [7]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [8]:
def preprocess(text):
    '''Function to preprocess English sentence'''
    text = text.lower() # lower casing
    text = re.sub("'", '', text) # remove the quotation marks if any
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.translate(remove_digits) # remove the digits
    text = text.strip()
    text = re.sub(" +", " ", text) # remove extra spaces
    text = '<start> ' + text + ' <end>'
    return text

In [9]:
def preprocess_kan(text):
    '''Function to preprocess Kannada sentence'''
    text = re.sub("'", '', text) # remove the quotation marks if any
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.strip()
    text = re.sub(" +", " ", text) # remove extra spaces
    text = '<start> ' + text + ' <end>'
    return text

In [10]:
df['English'] = df['English'].apply(preprocess)
df['Kannada'] = df['Kannada'].apply(preprocess_kan)


df.head()

,English,Kannada
0,<start> such videos are aplenty on social medi...,<start> ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು...
1,<start> this incident happened within the limi...,<start> ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದು...
2,<start> the situation is currently under contr...,<start> ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದ...
3,<start> choose a bookmark to be your home page...,<start> ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್...
4,<start> several social organisations are also ...,<start> ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರ...


In [11]:
def tokenize(lang):

  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post',maxlen=20,dtype='int32')

  return tensor, lang_tokenizer

In [12]:
def load_dataset():

  input_tensor, inp_lang_tokenizer = tokenize(df['English'].values)
  target_tensor, targ_lang_tokenizer = tokenize(df['Kannada'].values)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [13]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

In [14]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [15]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.15)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

212500 212500 37500 37500


In [16]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 128
units = 1024
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

vocab_inp_size =len(inp_lang.word_index.keys())
vocab_tar_size =len(targ_lang.word_index.keys())

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_inp_size+1, 300))
for word, i in inp_lang.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.300d.txt'

In [18]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding_layer_encoder",trainable=False)
        self.gru = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

                # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        x = self.embedding(x)
        
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        output, state = self.gru(x)
        
        output = tf.reshape(output, (-1, output.shape[2]))
        
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [20]:
tf.keras.backend.clear_session()

encoder = Encoder(vocab_inp_size+1, 300, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size+1, embedding_dim, units, BATCH_SIZE)

In [21]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [22]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    encoder.get_layer('embedding_layer_encoder').set_weights([embedding_matrix])
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 25

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

  print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')
  print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')